
# 🧠 Sentiment Analysis for E‑commerce Product Reviews

This notebook implements an **end‑to‑end solution** to automatically assess and categorize customer sentiments (**negative**, **neutral**, **positive**) from product reviews.

**What you'll get here:**
- Clean, reproducible pipeline (EDA → preprocessing → baselines → Transformer fine‑tuning → evaluation → error analysis).
- Ready-to-run code for both **classic ML** (TF‑IDF + Logistic/SVM) and **Transformer** (**DistilBERT**) models.
- Reusable **inference utilities** and an **optional FastAPI microservice** for deployment.

> Expected columns in your dataset CSV:
> - `Product ID` (string or int)
> - `Product Review` (text)
> - `Sentiment` (one of: `negative`, `neutral`, `positive`)


## 1) Setup

In [ ]:

# If you're on Colab or a fresh environment, uncomment the installs below.
# You can skip any line if the package already exists in your environment.

# %pip install -U pip
# %pip install pandas numpy scikit-learn matplotlib
# %pip install transformers==4.44.2 datasets==2.21.0 accelerate==0.34.2 evaluate==0.4.2
# %pip install torch --index-url https://download.pytorch.org/whl/cpu   # or proper CUDA wheel for your GPU
# %pip install fastapi uvicorn
# %pip install wordcloud emoji textblob


## 2) Imports

In [ ]:

import os
import re
import json
import math
import random
import numpy as np
import pandas as pd

from pathlib import Path
from collections import Counter

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import evaluate

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding,
    TrainingArguments, Trainer
)


## 3) Load Data

In [ ]:

# 🔁 EDIT: set the path to your CSV (with columns: Product ID, Product Review, Sentiment).
DATA_PATH = "reviews.csv"  # e.g., "data/ecomm_reviews.csv"

assert Path(DATA_PATH).exists(), f"CSV not found at {DATA_PATH}. Please update DATA_PATH."

df = pd.read_csv(DATA_PATH)

# Normalize expected column names (case-insensitive / extra spaces)
rename_map = {c: c.strip() for c in df.columns}
df.rename(columns=rename_map, inplace=True)

# Try to auto-detect reasonable column names if variations exist
def find_col(candidates):
    cols = {c.lower(): c for c in df.columns}
    for name in candidates:
        for col in df.columns:
            if col.lower().strip() == name.lower().strip():
                return col
    raise KeyError(f"Couldn't find any of: {candidates} in columns: {list(df.columns)}")

COL_ID = find_col(["Product ID", "product_id", "productid", "id"])
COL_TEXT = find_col(["Product Review", "review", "text", "comment", "content"])
COL_LABEL = find_col(["Sentiment", "label", "sentiment_label", "target"])

df = df[[COL_ID, COL_TEXT, COL_LABEL]].copy()
df.columns = ["product_id", "text", "label"]

df.head()


## 4) Exploratory Data Analysis

In [ ]:

# Basic stats
display(df.sample(min(5, len(df)), random_state=42))
print("Rows:", len(df))
print("Nulls:\n", df.isnull().sum())

# Drop rows with missing text or label
df = df.dropna(subset=["text", "label"]).copy()
df["text"] = df["text"].astype(str).str.strip()
df = df[df["text"].str.len() > 0]

# Class balance
print("\nClass distribution:")
print(df["label"].value_counts())

# Text length distribution
df["char_len"] = df["text"].str.len()
df["word_len"] = df["text"].str.split().str.len()

print("\nLength stats (chars):")
print(df["char_len"].describe())

print("\nLength stats (words):")
print(df["word_len"].describe())

# Plot length histogram (optional)
plt.figure()
df["word_len"].hist(bins=40)
plt.title("Word count distribution")
plt.xlabel("Words per review")
plt.ylabel("Frequency")
plt.show()


## 5) Preprocessing

In [ ]:

import emoji

def clean_text(s: str) -> str:
    s = s.lower()
    s = re.sub(r"http\S+|www\.\S+", " ", s)          # URLs
    s = re.sub(r"@\w+|#\w+", " ", s)                 # mentions/hashtags
    s = emoji.replace_emoji(s, replace=" ")          # remove/space emojis
    s = re.sub(r"[^a-z0-9\s\.\,\!\?]", " ", s)      # keep basic punctuations
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_clean"] = df["text"].map(clean_text)

# Map labels to ids (negative=0, neutral=1, positive=2)
label_set = sorted(df["label"].str.lower().unique().tolist())
label2id = {"negative":0, "neutral":1, "positive":2}
id2label = {v:k for k,v in label2id.items()}

# Validate labels
invalid = set(label_set) - set(label2id.keys())
if invalid:
    raise ValueError(f"Unexpected labels in data: {invalid}. Expected one of {list(label2id.keys())}")

df["y"] = df["label"].str.lower().map(label2id)
df.head(3)


## 6) Train / Validation / Test Split

In [ ]:

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["y"])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["y"])

for name, d in [("train", train_df), ("val", val_df), ("test", test_df)]:
    print(name, d.shape, Counter(d["y"]))


## 7) Classic ML Baselines (TF‑IDF + LR / SVM)

In [ ]:

X_train, y_train = train_df["text_clean"], train_df["y"]
X_val, y_val = val_df["text_clean"], val_df["y"]
X_test, y_test = test_df["text_clean"], test_df["y"]

# Pipeline: TFIDF + Logistic Regression
pipe_lr = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=2, max_df=0.9)),
    ("clf", LogisticRegression(max_iter=200, n_jobs=None))
])

# Simple grid for LR
param_grid_lr = {
    "clf__C": [0.5, 1.0, 2.0],
    "clf__penalty": ["l2"],
    "clf__solver": ["lbfgs"]
}
grid_lr = GridSearchCV(pipe_lr, param_grid_lr, cv=3, scoring="f1_macro", n_jobs=-1, verbose=1)
grid_lr.fit(X_train, y_train)

print("Best LR params:", grid_lr.best_params_)
val_pred_lr = grid_lr.predict(X_val)
print("\nValidation report (LR):\n", classification_report(y_val, val_pred_lr, target_names=[id2label[i] for i in [0,1,2]]))

# Pipeline: TFIDF + Linear SVM
pipe_svm = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=2, max_df=0.9)),
    ("clf", LinearSVC())
])
# Minimal grid for SVM
param_grid_svm = {"clf__C": [0.5, 1.0, 2.0]}
grid_svm = GridSearchCV(pipe_svm, param_grid_svm, cv=3, scoring="f1_macro", n_jobs=-1, verbose=1)
grid_svm.fit(X_train, y_train)

print("Best SVM params:", grid_svm.best_params_)
val_pred_svm = grid_svm.predict(X_val)
print("\nValidation report (SVM):\n", classification_report(y_val, val_pred_svm, target_names=[id2label[i] for i in [0,1,2]]))

# Pick the better baseline on val macro-F1
val_f1_lr = f1_score(y_val, val_pred_lr, average="macro")
val_f1_svm = f1_score(y_val, val_pred_svm, average="macro")
best_baseline, best_model = ("LR", grid_lr) if val_f1_lr >= val_f1_svm else ("SVM", grid_svm)
print(f"\nBest baseline: {best_baseline} (macro-F1={max(val_f1_lr, val_f1_svm):.4f})")


## 8) Transformer Fine‑tuning (DistilBERT)

In [ ]:

MODEL_NAME = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True)

train_ds = Dataset.from_pandas(train_df[["text", "y"]].rename(columns={"y":"labels"}))
val_ds   = Dataset.from_pandas(val_df[["text", "y"]].rename(columns={"y":"labels"}))
test_ds  = Dataset.from_pandas(test_df[["text", "y"]].rename(columns={"y":"labels"}))

train_ds = train_ds.map(tokenize_fn, batched=True)
val_ds   = val_ds.map(tokenize_fn, batched=True)
test_ds  = test_ds.map(tokenize_fn, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

id2label_map = {0:"negative", 1:"neutral", 2:"positive"}
label2id_map = {"negative":0, "neutral":1, "positive":2}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    id2label=id2label_map,
    label2id=label2id_map
)

metric_f1 = evaluate.load("f1")
metric_acc = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "f1_weighted": metric_f1.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }

training_args = TrainingArguments(
    output_dir="artifacts/distilbert-sentiment",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train();

# Evaluate on validation and test
val_metrics = trainer.evaluate(val_ds)
test_metrics = trainer.evaluate(test_ds)
print("Validation metrics:", val_metrics)
print("Test metrics:", test_metrics)


## 9) Evaluation & Error Analysis

In [ ]:

def evaluate_model(name, y_true, y_pred):
    print(f"\n=== {name} ===")
    print(classification_report(y_true, y_pred, target_names=[id2label[i] for i in [0,1,2]]))
    cm = confusion_matrix(y_true, y_pred, labels=[0,1,2])
    print("Confusion Matrix:\n", cm)
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "f1_weighted": f1_score(y_true, y_pred, average="weighted"),
    }

# Baseline on test
test_pred_bl = best_model.predict(test_df["text_clean"])
baseline_scores = evaluate_model(f"Baseline ({best_baseline})", test_df["y"], test_pred_bl)

# Transformer on test
tf_logits = trainer.predict(test_ds).predictions
tf_pred = tf_logits.argmax(axis=-1)
transformer_scores = evaluate_model("DistilBERT", test_df["y"], tf_pred)

print("\nSummary:")
print("Baseline:", baseline_scores)
print("DistilBERT:", transformer_scores)

# Show a few most confusing examples for the transformer
mis_idx = np.where(tf_pred != test_df["y"].values)[0].tolist()
print(f"\nMisclassified examples: {len(mis_idx)}")
for i in mis_idx[:10]:
    print("---")
    print("Text:", test_df.iloc[i]["text"][:500])
    print("True:", id2label[test_df.iloc[i]["y"]], "Pred:", id2label[tf_pred[i]])


## 10) Inference Utility

In [ ]:

def load_best_transformer():
    # Reload the best model saved by Trainer
    m = AutoModelForSequenceClassification.from_pretrained("artifacts/distilbert-sentiment/checkpoint-best", local_files_only=False)
    tok = AutoTokenizer.from_pretrained(MODEL_NAME)
    return m, tok

def predict_sentiment(texts, model=None, tok=None, batch_size=32):
    if model is None or tok is None:
        model, tok = load_best_transformer()
    enc = tok(texts, truncation=True, padding=True, return_tensors="pt")
    with torch.no_grad():
        logits = model(**enc).logits
    pred_ids = logits.argmax(dim=-1).tolist()
    return [id2label[i] for i in pred_ids]

# Example:
# predict_sentiment(["Loved it! Battery lasts long.", "Terrible quality, stopped working in a week."])


## 11) Save Baseline Model & Vectorizer

In [ ]:

import joblib
Path("artifacts/baseline").mkdir(parents=True, exist_ok=True)
joblib.dump(best_model, "artifacts/baseline/tfidf_baseline.joblib")
print("Saved baseline to artifacts/baseline/tfidf_baseline.joblib")


## 12) Optional: FastAPI Microservice (for deployment)

In [ ]:

FASTAPI_CODE = r'''
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

id2label = {0:"negative", 1:"neutral", 2:"positive"}

class Payload(BaseModel):
    texts: List[str]

app = FastAPI(title="Sentiment API", version="1.0.0")

MODEL_DIR = "artifacts/distilbert-sentiment/checkpoint-best"
TOKENIZER = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
tok = AutoTokenizer.from_pretrained(TOKENIZER)

@app.post("/predict")
def predict(payload: Payload):
    enc = tok(payload.texts, truncation=True, padding=True, return_tensors="pt")
    with torch.no_grad():
        logits = model(**enc).logits
    preds = logits.argmax(dim=-1).tolist()
    labels = [id2label[i] for i in preds]
    return {"predictions": labels}
'''

Path("artifacts/api").mkdir(parents=True, exist_ok=True)
with open("artifacts/api/app.py", "w", encoding="utf-8") as f:
    f.write(FASTAPI_CODE)

print("FastAPI app written to artifacts/api/app.py")
print("Run locally with:")
print("  uvicorn artifacts.api.app:app --reload --port 8000")



## 13) Notes & Tips
- Start with the **baseline** model and verify you have stable metrics, then fine‑tune **DistilBERT**.
- Use **macro‑F1** to trade off class imbalance. Consider up/down‑sampling if classes are skewed.
- For small datasets, add **data augmentation** (e.g., back‑translation, synonym replacement).
- Keep an eye on **leakage** (no duplicates across splits).
- Log experiments and **seed** randomness for reproducibility.
